In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import mlflow
import mlflow.sklearn

# Função para leitura do dataset
def leitura_dataset(path):
    df = pd.read_csv(path)
    return df

# Caminho para o dataset
diabete_csv_path = "dataset/diabetes.csv"
diabete_csv = leitura_dataset(diabete_csv_path)

# Separar as features e o rótulo
X = diabete_csv.drop('Outcome', axis=1)
y = diabete_csv['Outcome']

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hiperparâmetros para GridSearch
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Iniciar o MLFlow
try:
    # Encerra a execução anterior, se houver
    mlflow.end_run()
    
    with mlflow.start_run(run_name='Diabetes_modelo'):
        # Criar o classificador
        rf = RandomForestClassifier()
    
        # Realizar o GridSearch
        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
    
        # Melhor modelo
        best_model = grid_search.best_estimator_
    
        # Fazer previsões
        predictions = best_model.predict(X_test)

        # Avaliar o modelo
        print(confusion_matrix(y_test, predictions))
        print(classification_report(y_test, predictions))

        # Criar um exemplo de entrada
        input_example = np.array([[5, 166, 72, 19, 175, 25.8, 0.587, 51]])  # Exemplo com valores fictícios

        # Registrar o modelo com MLFlow, incluindo o exemplo de entrada
        mlflow.sklearn.log_model(best_model, "model", input_example=input_example)
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metric("accuracy", grid_search.best_score_)

        # Salvar o modelo localmente usando joblib
        joblib.dump(best_model, 'modelo_diabetes.pkl')

except Exception as e:
    print(f"Ocorreu um erro: {e}")

finally:
    # Carregar o modelo salvo
    loaded_model = joblib.load('modelo_diabetes.pkl')

    # Testar o modelo com novos dados
    test_data = np.array([[5, 166, 72, 19, 175, 25.8, 0.587, 51]])
    predicted = loaded_model.predict(test_data)

    print(f'Predição: {predicted}')


[[82 17]
 [17 38]]
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        99
           1       0.69      0.69      0.69        55

    accuracy                           0.78       154
   macro avg       0.76      0.76      0.76       154
weighted avg       0.78      0.78      0.78       154



/usr/local/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Predição: [1]


/usr/local/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
